In [28]:
from medoid_lib import api
from medoid_lib.dtw_lib._dtw_lib import fastdtw
f = lambda x, y: fastdtw(x, y, 1)[0]

In [30]:
limit = 50
all_activities = api.load(['medoid/data/all_activities'])[0]
check_limit = lambda x, y: len(api.load(['medoid/data/user' + str(y) + '_' + x + '_matrix_5'])[0]) > limit
user1_activity = list(filter(lambda x: check_limit(x, 1), all_activities[1]))
user2_activity = list(filter(lambda x: check_limit(x, 2), all_activities[2]))

In [36]:
user1_matrix_name = list(map(lambda x: 'medoid/data/user1_' + x + '_matrix_5', user1_activity))
user1_segs_name = list(map(lambda x: 'medoid/data/user1_' + x + '_segments_5', user1_activity))
user1_matrices = api.load(user1_matrix_name)
user1_segs = api.load(user1_segs_name)
user2_matrix_name = list(map(lambda x: 'medoid/data/user2_' + x + '_matrix_5', user2_activity))
user2_segs_name = list(map(lambda x: 'medoid/data/user2_' + x + '_segments_5', user2_activity))
user2_matrices = api.load(user2_matrix_name)
user2_segs = api.load(user2_segs_name)

In [62]:
get_multi = lambda x, y: [v for idx, v in enumerate(x) if idx in y]

def form_new_matrices(indices, matrix):
    length = len(indices)
    new_matrix = get_multi(matrix, indices)
    new_matrix = list(map(lambda x: get_multi(x, indices), new_matrix))
    return new_matrix

def split_data(matrix, segs, prob):
    import random
    size = len(matrix)
    indices = list(range(size))
    test_indices = random.sample(indices, int(prob * size))
    train_indices = [i for i in indices if i not in test_indices]
    test_segs = get_multi(segs, test_indices)
    test_matrix = form_new_matrices(test_indices, matrix)
    train_segs = get_multi(segs, train_indices)
    train_matrix = form_new_matrices(train_indices, matrix)
    return (train_matrix, train_segs), (test_matrix, test_segs)

In [70]:
user1_data = list(map(lambda x: split_data(x[0], x[1], 0.2), zip(user1_matrices, user1_segs)))
user1_train = [x[0] for x in user1_data]
user1_train_matrix, user1_train_seg = [], []
for i in user1_train:
    user1_train_matrix.append(i[0])
    user1_train_seg.append(i[1])

In [76]:
user1_represents = api.get_multi_represents(user1_train_matrix, user1_train_seg, user1_activity, f)

In [104]:
user1_test_segs = []
user1_test_labels = []
all_segs = [x[1][1] for x in user1_data]
for i in range(len(all_segs)):
    user1_test_segs.extend(all_segs[i])
    user1_test_labels.extend([user1_activity[i] for _ in range(len(all_segs[i]))])
user1_test_data = list(zip(user1_test_segs, user1_test_labels))
random.shuffle(user1_test_data)
user1_test_segs = [x[0] for x in user1_test_data]
user1_test_labels = [x[1] for x in user1_test_data]

In [110]:
api.evaluate(user1_test_segs, user1_test_labels, user1_represents, f, top=1)

0.10550458715596331

In [119]:
import importlib
importlib.reload(api)

<module 'medoid_lib.api' from '/home/t/tianyang/curantis/medoid/medoid_lib/api.py'>

In [120]:
api.evaluate(user1_test_segs, user1_test_labels, user1_represents, f, top=3)

100%|██████████| 436/436 [03:53<00:00,  1.85it/s]


0.44036697247706424

In [121]:
api.evaluate(user1_test_segs, user1_test_labels, user1_represents, f, top=5)

100%|██████████| 436/436 [03:53<00:00,  1.85it/s]


0.5573394495412844

In [108]:
api.classify(user1_represents, user1_test_segs, f, top = 6)

['Working on laptop',
 'Walking',
 'Sitting',
 'Walking',
 'Commute in bus',
 'Eating using fork and spoon']

In [125]:
user2_data = list(map(lambda x: split_data(x[0], x[1], 0.2), zip(user2_matrices, user2_segs)))
user2_train = [x[0] for x in user2_data]
user2_train_matrix, user2_train_seg = [], []
for i in user2_train:
    user2_train_matrix.append(i[0])
    user2_train_seg.append(i[1])
user2_represents = api.get_multi_represents(user2_train_matrix, user2_train_seg, user2_activity, f)
user2_test_segs = []
user2_test_labels = []
all_segs = [x[1][1] for x in user2_data]
for i in range(len(all_segs)):
    user2_test_segs.extend(all_segs[i])
    user2_test_labels.extend([user2_activity[i] for _ in range(len(all_segs[i]))])
user2_test_data = list(zip(user2_test_segs, user2_test_labels))
random.shuffle(user2_test_data)
user2_test_segs = [x[0] for x in user2_test_data]
user2_test_labels = [x[1] for x in user2_test_data]

In [126]:
api.evaluate(user2_test_segs, user2_test_labels, user2_represents, f, top=1)


  0%|          | 0/617 [00:00<?, ?it/s]/home/t/tianyang/curantis/medoid/medoid_lib/api.py:255: RuntimeWarning: divide by zero encountered in double_scalars
  dis_f = lambda rep: distance(rep[0], seg) / rep[1]

 25%|██▍       | 152/617 [01:16<03:57,  1.96it/s]


 50%|████▉     | 306/617 [02:35<02:46,  1.87it/s]


 75%|███████▍  | 460/617 [03:53<01:19,  1.97it/s]


100%|█████████▉| 614/617 [05:11<00:01,  1.97it/s]


100%|██████████| 617/617 [05:13<00:00,  1.98it/s]

0.20907617504051865

In [127]:
api.evaluate(user2_test_segs, user2_test_labels, user2_represents, f, top=3)


 25%|██▌       | 155/617 [01:18<03:52,  1.99it/s]


 50%|█████     | 309/617 [02:36<02:34,  1.99it/s]


 75%|███████▌  | 463/617 [03:54<01:18,  1.97it/s]


100%|██████████| 617/617 [05:11<00:00,  1.99it/s]

0.7893030794165316

In [128]:
api.evaluate(user2_test_segs, user2_test_labels, user2_represents, f, top=5)


 25%|██▌       | 155/617 [01:18<03:52,  1.98it/s]


 50%|█████     | 309/617 [02:36<02:34,  1.99it/s]


 75%|███████▌  | 463/617 [03:54<01:18,  1.97it/s]


100%|██████████| 617/617 [05:12<00:00,  1.99it/s]

0.9773095623987034